In [ ]:
#after training the model, we can use this file to test the model. RUn this notebook in A100 GPU machine (NC24adsA100 compute instance) with 

In [ ]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
import time
from azure.ai.ml import MLClient, Input
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
credential = DefaultAzureCredential()
subscription_id = "" # your subscription id
resource_group = ""#your resource group
workspace = "" #your workspace name
workspace_ml_client = MLClient(credential, subscription_id, resource_group, workspace)

In [ ]:
model_name = "llama2_13b_fine_tuned"
model_path="./"
workspace_ml_client.models.download(model_name, version="2",download_path=model_path)
#after this step, remove the redundant parent folder name "llama2_13b_fine_tuned" so that the downloaded folder only has one 

In [ ]:
import mlflow
import pandas as pd
model = mlflow.pyfunc.load_model(model_name)
prompt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a chessboard with the given size and with pieces at the specified positions.
### Input:
Size: 8x8 \nPiece positions:\nWhite Rook at H8\nWhite Pawn at B3\nBlack Pawn at A3\nBlack Rook at H2

"""
model_input= {"text":[prompt], "max_length":100}
model.predict(model_input)


## Deploy to managed online endpoint and test

1. Create online endpoint: ```az ml online-endpoint create -f deployment/endpoint.yml```
2. Create the deployment: ```az ml online-deployment update -f deployment/deployment.yml```

In [ ]:
## Test

In [ ]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
prompt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a chessboard with the given size and with pieces at the specified positions.
### Input:
Size: 8x8 \nPiece positions:\nWhite Rook at H8\nWhite Pawn at B3\nBlack Pawn at A3\nBlack Rook at H2
"""
data= {"data":{"text":[prompt], "max_length":100}}

body = str.encode(json.dumps(data))

url = ''
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = ''
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'blue' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))